In [3]:
#INSTALL LIBRARIES------------------------------------------
!pip install transformers scikit-learn datasets wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
    

In [4]:
#IMPORTS-----------------------------
from pprint import pprint
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaModel, AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from torch.nn import Linear, ReLU
import pdb
import numpy as np, torch, random as rnd, torch.nn as nn, wandb
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import sys, os, json
from transformers import AutoModelForQuestionAnswering
from torch.nn.functional import cosine_similarity
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

from transformers import AutoTokenizer
import os, time



In [5]:


# adversarial = load_dataset("iperbole/adversarial_fever_nli")["test"]

# ds = load_dataset("tommasobonomo/sem_augmented_fever_nli")

# training_set = ds["train"]

# validation_set = ds["validation"]

# test_set = ds["test"]

# pprint(adversarial[0])
# pprint(training_set[0])

In [6]:

# import nltk
# from nltk.corpus import stopwords

# nltk.download('stopwords')
# print(stopwords.words('english'))



In [7]:
# f1 = "test per vedere come va"
# f2 = "questa è una prova"

# tokenized = tokenizer(f1+ tokenizer.eos_token + f2, return_tensors='pt', padding='max_length', max_length=40, return_token_type_ids=True)

# print(tokenized)
# print(tokenized["input_ids"].shape)
# print(type(tokenized))

# out = model(**tokenized)
# print(out['last_hidden_state'].shape)

# print(out.last_hidden_state.mean(dim=-1).squeeze().shape)

In [8]:
# Function to print a progress bar
def print_progress_bar(percentuale: float, lunghezza_barra: int = 30, text: str="") -> None:
    blocchi_compilati = int(lunghezza_barra * percentuale)
    barra = "[" + "=" * (blocchi_compilati - 1) + ">" + " " * (lunghezza_barra - blocchi_compilati) + "]"
    sys.stdout.write(f"\r{barra} {percentuale * 100:.2f}% complete " + text)
    sys.stdout.flush()

In [9]:
class NLIDataset(Dataset):



    def __init__(self, data, file_name, adversarial = False):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.encode_labels = {'CONTRADICTION': 0, 'NEUTRAL': 1, 'ENTAILMENT': 2}
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        self.model = RobertaModel.from_pretrained('roberta-base').to(self.device)
        self.distilbert_tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
        self.file_name = file_name
        self.adversarial = adversarial
        self.preprocess_function(data)
        self.tokenizer = None
        self.model = None



    def preprocess_function(self, examples):
        premises = []
        answers = []
        hypothesis = []
        similarities = {}
        ordered_similarities = []
        file_exists = os.path.isfile("data/" + self.file_name)
        if file_exists:
            with open("data/" + self.file_name, "r") as f:
                similarities = json.load(f)
        # Utilizza un ciclo for per popolare le tre liste
        #breakpoint()
        for i,example in enumerate(examples):

            print_progress_bar(i / len(examples), text=" | preprocessing")
            premises.append(example["premise"].strip())
            answers.append(self.encode_labels[example["label"]] )
            hypothesis.append(example["hypothesis"].strip())
            if not file_exists:
                s1 = self.embed_sentence(example["premise"].strip())
                s2 = self.embed_sentence(example["hypothesis"].strip())
                if self.adversarial:


                    similarities[str(example["cid"]) + example["hypothesis"]] = cosine_similarity(s1, s2).item()
                    ordered_similarities.append(similarities[str(example["cid"]) + example["hypothesis"]])
                else:
                    similarities[example["id"]] = cosine_similarity(s1, s2).item()
                    ordered_similarities.append(similarities[example["id"]])
            elif self.adversarial:
                ordered_similarities.append(similarities[str(example["cid"]) + example["hypothesis"]])
            else:
                ordered_similarities.append(similarities[example["id"]])
        #breakpoint()
        inputs = self.distilbert_tokenizer(
            hypothesis,
            premises,
            max_length=384,
            truncation="only_second",
            return_offsets_mapping=True,
            padding="max_length",
            return_tensors="pt"
        )
        if not file_exists:
            if not os.path.exists("data"):
                os.makedirs("data")
            with open("data/" + self.file_name, "w") as f:
                json.dump(similarities, f, indent=4)
        inputs["label"] = torch.tensor(answers)
        inputs["similarity"] = torch.tensor(ordered_similarities)
        self.data = inputs


    def embed_sentence(self, sentence):
        # Tokenizza la frase
        inputs = self.tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
        # Ottieni gli embedding dal modello
        with torch.no_grad():
            outputs = self.model(**inputs.to(self.device))
        # Usa l'output del modello come embedding (puoi usare altri livelli o combinazioni se preferisci)
        # Prendi il vettore medio (puoi anche scegliere il vettore della [CLS] token, ecc.)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings

    def __len__(self):
        return len(self.data["input_ids"])

    def __getitem__(self, idx):
        return self.data["input_ids"][idx], self.data["attention_mask"][idx], self.data["label"][idx], self.data["similarity"][idx]

    def collate(self, batch):
        #breakpoint()
        x = []
        attention_mask = []
        y = []
        z = []
        for x_batch, attention_mask_batch, y_batch, z_batch in batch:
            x.append(x_batch)
            attention_mask.append(attention_mask_batch)
            y.append(y_batch)
            z.append(z_batch)

        x = torch.stack(x)
        attention_mask = torch.stack(attention_mask)
        y = torch.stack(y)
        z = torch.stack(z)
        #breakpoint()
        # x = pad_sequence(x, batch_first=True)
        # attention_mask = pad_sequence(attention_mask, batch_first=True)
        return x.to(self.device),attention_mask.to(self.device), y.to(self.device), z.to(self.device)



    def get_dataloader(self, batch_size):
        return DataLoader(self, batch_size=batch_size, shuffle=True, collate_fn = self.collate)

















In [10]:
# prompt: create a torch model using roberta and a linear layer

class RobertaClassifier(nn.Module):
  def __init__(self, use_similarity, num_labels=3):
    super(RobertaClassifier, self).__init__()
    self.distilbert =  AutoModel.from_pretrained("distilbert/distilbert-base-uncased")
    self.linear = nn.Linear(self.distilbert.config.hidden_size+1, num_labels)
    self.use_similarity = use_similarity

  def freeze(self, epoch):
    if epoch == 2:
        freeze_until_layer = len(self.distilbert.parameters())
    else:
        freeze_until_layer = None  # Non congela nulla

    # Congela i layer di DistilBERT
    if freeze_until_layer is not None:
        for idx, param in enumerate(self.distilbert.parameters()):
            if idx < freeze_until_layer:
                param.requires_grad = False
            else:
                param.requires_grad = True



  def forward(self, input_ids, attention_mask, similarities):

   # breakpoint()
    if not self.use_similarity:
        similarities = torch.zeros(input_ids.shape[0]).to('cuda' if torch.cuda.is_available() else 'cpu')

    outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
    #breakpoint()

    pooled_output = torch.cat((outputs.last_hidden_state[:, 0, :], similarities.unsqueeze(1)), dim=1)
    # outputs.last_hidden_state.mean(dim=-1)

    logits = self.linear(pooled_output)

    return logits


In [23]:


class Trainer():

    def __init__(self, model,train_dataloader, validation_dataloader, optimizer, loss_function, device, test_dataloader=None):
        self.model = model.to(device)
        self.train_dataloader = train_dataloader
        self.validation_dataloader = validation_dataloader
        self.optimizer = optimizer
        self.loss_function = loss_function
        self.device = device
        self.test_dataloader = test_dataloader


    @staticmethod
    def evaluation_parameters(y_true, y_pred):
        #breakpoint()
        y_pred = np.argmax(y_pred, axis=1)
        cm = confusion_matrix(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average='weighted')
        recall = recall_score(y_true, y_pred, average='weighted')
        f1 = f1_score(y_true, y_pred, average='weighted')
        accuracy = accuracy_score(y_true, y_pred)
        return cm, precision, recall, f1, accuracy

    @staticmethod
    def format_time_delay(seconds):
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        seconds = seconds % 60
        return hours, minutes, seconds


    def train(self, epochs: int, use_wandb: bool = False, config: dict = {}, name: str="", target_f1: float=0.0):
        start_time = time.time()
        best_model = None
        save = False
        if use_wandb:
            wandb.init(
                # Set the project where this run will be logged
                project="nlphw2",
                name=name,
                # Track hyperparameters and run metadata
                config=config
            )
        validation_loss, precision, recall, f1, accuracy = self.validate(use_wandb)
        total_loss = validation_loss
        if use_wandb:
                wandb.log({"validation_loss": validation_loss,
                      "precision": precision,
                      "recall": recall,
                      "f1": f1,
                      "accuracy": accuracy,
                      "train_loss": total_loss / len(self.train_dataloader)})
        for epoch in range(epochs):

            time_delay = time.time() - start_time
            hours, minutes, seconds = format_time_delay(time_delay_seconds)
            print(f"\nTempo trascorso: {hours} ore, {minutes} minuti, {seconds} secondi")
            self.model.freeze(epoch)
            self.model.train()  # Set the model to training mode
            total_loss = 0
            #breakpoint()
            for i, batch in enumerate(self.train_dataloader):
                print_progress_bar(i / len(self.train_dataloader), text=f" | training epoch {epoch}")
                # Get the inputs and targets from the batch
                inputs, mask, targets, similarities = batch

                # Zero the gradients
                self.optimizer.zero_grad()
                # Forward pass
                outputs = self.model(inputs, mask, similarities)
                #print("outputs = ", outputs,"\ntargets = ", targets)
                #breakpoint()
                # Compute loss
                loss = self.loss_function(outputs, targets)
                #print(loss)
                # Backward pass and optimize
                loss.backward()
                self.optimizer.step()
                # Accumulate the total loss
                total_loss += loss.item()

            # Print the average loss for this epoch
            validation_loss, precision, recall, f1, accuracy = self.validate(use_wandb)
            if f1 > target_f1:
                best_model = self.model.state_dict()
                target_f1 = f1
                save = True
            if use_wandb:
                wandb.log({"validation_loss": validation_loss,
                      "precision": precision,
                      "recall": recall,
                      "f1": f1,
                      "accuracy": accuracy,
                      "train_loss": total_loss / len(self.train_dataloader)})
        if save:
            torch.save(best_model, name + f'-{target_f1}.pth')
        print(target_f1)
        if use_wandb:
            wandb.finish()



    def validate(self, use_wandb: bool = False, test=False):
        dataloader = self.test_dataloader if test else self.validation_dataloader
        if dataloader is None:
            print("empty dataloader!")
            exit(1)
        self.model.eval()  # Set the model to evaluation mode
        total_loss = 0
        all_predictions = torch.tensor([])
        all_targets = torch.tensor([])
        with torch.no_grad():  # Do not calculate gradients
            for i, batch in enumerate(self.validation_dataloader):
                print_progress_bar(i / len(dataloader), text=" | validation")
                # Get the inputs and targets from the batch
                inputs, mask, targets, similarities  = batch

                # Forward pass
                outputs = self.model(inputs, mask, similarities)
                # Compute loss
                #breakpoint()
                loss = self.loss_function(outputs, targets)
                # Accumulate the total loss
                total_loss += loss.item()
                # Store predictions and targets
                all_predictions = torch.cat((all_predictions, outputs.squeeze().round().cpu()))
                all_targets = torch.cat((all_targets, targets.cpu()))
        validation_loss = total_loss / len(self.validation_dataloader)
        #breakpoint()
        cm, precision, recall, f1, accuracy = self.evaluation_parameters(all_targets, all_predictions)
        return validation_loss, precision, recall, f1, accuracy




In [12]:
class BaselineStratifiedModel(nn.Module):

    def __init__(self, len0, len1):
        super(BaselineStratifiedModel, self).__init__()
        self.p = len0/(len0+len1)

    def forward(self, x):
        return torch.tensor([0 if np.random.rand() < self.p else 1 for _ in range(x[0].shape[0])], dtype=torch.float)



In [13]:
new_seed = 108
def set_seed(seed):
    np.random.seed(seed)
    rnd.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True # Se stai usando GPU
    return seed, seed+1

In [15]:


# Imposta il seed per avere riproducibilità

seed, new_seed = set_seed(new_seed)

In [16]:

adversarial = load_dataset("iperbole/adversarial_fever_nli")["test"]

ds = load_dataset("tommasobonomo/sem_augmented_fever_nli")

training_set = ds["train"]

validation_set = ds["validation"]

test_set = ds["test"]

wandb.login(key='aaf831dabc88d936d4e6b439b798bb4cb42814ea')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/337 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/51086 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2288 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2287 [00:00<?, ? examples/s]

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
!git clone https://github.com/monteleone-1883922/hw2_nlp.git
os.chdir("hw2_nlp")
!git checkout huggingFaceBase
!mv data ./../data
os.chdir("..")
!rm -rf hw2_nlp


Cloning into 'hw2_nlp'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (74/74), done.
Receiving objects: 100% (78/78), 726.37 KiB | 10.53 MiB/s, done.
remote: Total 78 (delta 44), reused 14 (delta 3), pack-reused 0
Resolving deltas: 100% (44/44), done.
Branch 'huggingFaceBase' set up to track remote branch 'huggingFaceBase' from 'origin'.
Switched to a new branch 'huggingFaceBase'


In [18]:
train_dataset = NLIDataset(training_set, "train_similarities.json")

validation_dataset = NLIDataset(validation_set, "validation_similarities.json")

test_dataset = NLIDataset(test_set, "test_similarities.json")

adversarial_dataset = NLIDataset(adversarial, "adversarial_similarities.json", adversarial=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

[============================> ] 99.96% complete  | preprocessing

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[============================> ] 99.96% complete  | preprocessing

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[============================> ] 99.70% complete  | preprocessing

In [19]:
# prompt: genera i dataloader

train_dataloader = train_dataset.get_dataloader(batch_size=50)
validation_dataloader = validation_dataset.get_dataloader(batch_size=50)
test_dataloader = test_dataset.get_dataloader(batch_size=32)
adversarial_dataloader = adversarial_dataset.get_dataloader(batch_size=32)


In [24]:

model = RobertaClassifier(False)

trainer = Trainer(model, train_dataloader, validation_dataloader, torch.optim.Adam(model.parameters(), lr=1e-5), nn.CrossEntropyLoss(),'cuda' if torch.cuda.is_available() else 'cpu', test_dataloader)



In [21]:
trainer.validate(test=False)

[============================> ] 97.83% complete  | validation

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(1.107908668725387,
 0.23084976293831394,
 0.33916083916083917,
 0.18635372958499985,
 0.33916083916083917)

In [25]:

trainer.train(6, use_wandb=True, name="base_1e-5_"+ str(seed))




[============================> ] 97.83% complete  | validation

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NameError: name 'format_time_delay' is not defined

In [ ]:
# del training_set

# dataset = NLIDataset(validation_set)


# dataloader = dataset.get_dataloader(batch_size=32, pos_num=1)

# model = RobertaClassifier()

# batch = next(iter(dataloader))

# model(batch[0], batch[1], batch[3])